In [1]:
import pandas as pd
import os
import json

In [2]:
# Set the search path for files (assuming the directory is relative to the current script)
file_path_mc010 = os.path.join("..", "fresh_data_merging_scripts_files", "merge_mc010_fresh_data.csv")

df = pd.read_csv(file_path_mc010)

df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13208 entries, 0 to 13247
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   datetime   13208 non-null  object 
 1   station    13208 non-null  object 
 2   core       13208 non-null  object 
 3   component  13208 non-null  object 
 4   period     13208 non-null  object 
 5   value      13208 non-null  float64
dtypes: float64(1), object(5)
memory usage: 722.3+ KB


In [3]:
# remove null values
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13208 entries, 0 to 13247
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   datetime   13208 non-null  object 
 1   station    13208 non-null  object 
 2   core       13208 non-null  object 
 3   component  13208 non-null  object 
 4   period     13208 non-null  object 
 5   value      13208 non-null  float64
dtypes: float64(1), object(5)
memory usage: 722.3+ KB


In [4]:
# remove timezone information
df.loc[:, 'datetime'] = df['datetime'].astype(str).str.slice(0, 19)
df['datetime'] = pd.to_datetime(df['datetime'], format='mixed')
df.loc[:, 'datetime'] = df['datetime'].dt.tz_localize(None)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13208 entries, 0 to 13247
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   datetime   13208 non-null  datetime64[ns]
 1   station    13208 non-null  object        
 2   core       13208 non-null  object        
 3   component  13208 non-null  object        
 4   period     13208 non-null  object        
 5   value      13208 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 722.3+ KB


In [5]:
# split date information from datetime
df['hour'] = df['datetime'].dt.strftime('%H')  # Hour (00-23)
df['day'] = df['datetime'].dt.strftime('%d')  # Day of the month (01-31)
df['month'] = df['datetime'].dt.strftime('%m')  # Month (01-12)
df['year'] = df['datetime'].dt.strftime('%Y')  # Month (01-12)
df.sample(3)

,datetime,station,core,component,period,value,hour,day,month,year
3419,2024-06-07 06:00:00,mc010,o3,o3_1h,1h,42.0,06,07,06,2024
6207,2024-07-18 21:00:00,mc010,no,no_1h,1h,2.0,21,18,07,2024
6595,2024-07-16 04:00:00,mc010,pm2,pm2_1h,1h,10.0,04,16,07,2024


In [6]:
# add day of the week to dataframe
import calendar

days = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}

df_prepped = df
# convert the 'day', 'month', and 'year' columns to integers
df_prepped['day'] = df['day'].astype(int)
df_prepped['month'] = df['month'].astype(int)
df_prepped['year'] = df['year'].astype(int)

# function to determine the day of the week
def get_day_of_week(row):
    return calendar.weekday(row['year'], row['month'], row['day'])

# function to determine if day is weekday or weekend 
def is_weekend(day_number):
    return 1 if day_number >= 5 else 0
    #day_number >= 5 ? 1 : 0
    
# apply the functions to create the new columns
df_prepped['day_of_week'] = df_prepped.apply(get_day_of_week, axis=1)
df_prepped['is_weekend'] = df_prepped['day_of_week'].apply(is_weekend)

df_prepped.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13208 entries, 0 to 13247
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   datetime     13208 non-null  datetime64[ns]
 1   station      13208 non-null  object        
 2   core         13208 non-null  object        
 3   component    13208 non-null  object        
 4   period       13208 non-null  object        
 5   value        13208 non-null  float64       
 6   hour         13208 non-null  object        
 7   day          13208 non-null  int32         
 8   month        13208 non-null  int32         
 9   year         13208 non-null  int32         
 10  day_of_week  13208 non-null  int64         
 11  is_weekend   13208 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int32(3), int64(2), object(5)
memory usage: 1.2+ MB


In [7]:
#------------- prep -------------

# get h-1 for all core values
dfs = {}

for value in df['core'].unique():
    filtered_df = df_prepped[df_prepped['core'] == value]
    dfs[value] = filtered_df

df_no2 = dfs['no2']
df_no = dfs['no']
df_nox = dfs['nox']
df_pm10 = dfs['pm10']
df_pm2 = dfs['pm2']

#------------- no2 -------------

# get no2 for h-1 values
df_no2 = df_no2.copy()
df_no2 = df_no2.rename(columns={'value': 'no2'})
df_no2.loc[:, 'no2_h-1'] = df_no2.groupby(['station', 'core'])['no2'].shift(-1)

# remove unnecessary columns which will lead to merge duplicates
df_no2 = df_no2.drop(columns=['core'])

#------------- no -------------

# get no for h-1 values
df_no = df_no.copy()
df_no = df_no.rename(columns={'value': 'no'})
df_no.loc[:, 'no_h-1'] = df_no.groupby(['station', 'core'])['no'].shift(-1)

# remove unnecessary columns which will lead to merge duplicates
df_no = df_no.loc[:, ['datetime','no', 'no_h-1']]

#------------- nox -------------

# get nox for h-1 values
df_nox = df_nox.copy()
df_nox = df_nox.rename(columns={'value': 'nox'})
df_nox.loc[:, 'nox_h-1'] = df_nox.groupby(['station', 'core'])['nox'].shift(-1)

# remove unnecessary columns which will lead to merge duplicates
df_nox = df_nox.loc[:, ['datetime','nox', 'nox_h-1']]

#------------- pm10 -------------

# get pm10 for h-1 values
df_pm10 = df_pm10.copy()
df_pm10 = df_pm10.rename(columns={'value': 'pm10'})
df_pm10.loc[:, 'pm10_h-1'] = df_pm10.groupby(['station', 'core'])['pm10'].shift(-1)

# remove unnecessary columns which will lead to merge duplicates
df_pm10 = df_pm10.loc[:, ['datetime','pm10', 'pm10_h-1']]

#------------- pm2.5 -------------

df_pm2 = df_pm2.copy()
df_pm2 = df_pm2.rename(columns={'value': 'pm2.5'})
df_pm2.loc[:, 'pm2.5_h-1'] = df_pm2.groupby(['station', 'core'])['pm2.5'].shift(-1)

# remove unnecessary columns which will lead to merge duplicates
df_pm2 = df_pm2.loc[:, ['datetime', 'pm2.5', 'pm2.5_h-1']]

#------------- merge & order -------------

# create list of dataframes
dataframes_l = [df_pm2, df_pm10, df_nox, df_no, df_no2]

# get the first df
df_m = dataframes_l[0]

# iterate through the dataframes
for df in dataframes_l[1:]:
    # perform an outer join on 'datetime'
    df_m = pd.merge(df_m, df, on='datetime', how='outer')

# Sort the merged DataFrame by 'datetime'
df_m = df_m.sort_values(by='datetime')

# reindexing the columns (change column order)
column_order_new = ['datetime',  'station', 'hour', 'day', 'month', 'year', 'day_of_week', 'is_weekend', 
                    'no2', 'no', 'nox', 'pm10', 'pm2.5', 'no2_h-1', 'no_h-1', 'nox_h-1', 'pm10_h-1', 'pm2.5_h-1']
df_m = df_m.reindex(columns=column_order_new)

df_m.head()

,datetime,station,hour,day,month,year,day_of_week,is_weekend,no2,no,nox,pm10,pm2.5,no2_h-1,no_h-1,nox_h-1,pm10_h-1,pm2.5_h-1
716,2024-06-01 00:00:00,mc010,00,1.0,6.0,2024.0,5.0,1.0,13.0,2.0,14.0,13.0,7.0,16.0,2.0,16.0,21.0,11.0
715,2024-06-01 01:00:00,mc010,01,1.0,6.0,2024.0,5.0,1.0,14.0,2.0,16.0,13.0,10.0,13.0,2.0,14.0,13.0,7.0
714,2024-06-01 02:00:00,mc010,02,1.0,6.0,2024.0,5.0,1.0,12.0,2.0,14.0,17.0,13.0,14.0,2.0,16.0,13.0,10.0
713,2024-06-01 03:00:00,mc010,03,1.0,6.0,2024.0,5.0,1.0,12.0,2.0,13.0,20.0,15.0,12.0,2.0,14.0,17.0,13.0
712,2024-06-01 04:00:00,mc010,04,1.0,6.0,2024.0,5.0,1.0,9.0,2.0,11.0,22.0,16.0,12.0,2.0,13.0,20.0,15.0


In [9]:
df_m.to_csv("mc010_fresh_prepped.csv", index=False)